In [ ]:
## installing packages
!python3 -m pip install -q "mxnet<2.0.0"
!python3 -m pip install -q autogluon
!python3 -m pip install -q graphviz --upgrade

In [ ]:
# imports
import gc
import os
import shutil
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
## define configuration
PATH_TRAIN = '../input/tabular-playground-series-nov-2021/train.csv'
PATH_TEST = '../input/tabular-playground-series-nov-2021/test.csv'

time_limit = 20000 # (in secs. for train run: set as 300 which gave public score ~10)

In [ ]:
train = pd.read_csv(PATH_TRAIN, index_col="id")
test = pd.read_csv(PATH_TEST, index_col="id")

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64','float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                else:
                    df[col] = df[col].astype(np.float32)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
train.head()

In [ ]:
## imports
from autogluon.tabular import TabularPredictor, TabularDataset

In [ ]:
## run model

model_ag = TabularPredictor(label='target')

model_ag.fit(train_data=train, 
              time_limit=time_limit, 
              presets='best_quality',
              num_stack_levels = 3,
              num_bag_folds = 5,
              num_bag_sets = 1,
              excluded_model_types = ['KNN', 'LR'],)

del train['target']

In [ ]:
## check leaderboard
model_ag.leaderboard()

In [ ]:
preds_autogluon = model_ag.predict_proba(TabularDataset(test))
preds_autogluon

In [ ]:
## create submission
sub = pd.read_csv("../input/tabular-playground-series-nov-2021/sample_submission.csv")

submission = pd.DataFrame({
    'id': preds_autogluon.index,
    'target': preds_autogluon[1]
})

submission.head()

In [ ]:
## save submission
PATH_AUTOGLUON_SUBMISSION = 'submission_autogluon_' + str(time_limit) + '.csv'
PATH_AUTOGLUON_SUBMISSION

In [ ]:
submission.to_csv(PATH_AUTOGLUON_SUBMISSION, index=False)

In [ ]:
## clear memory
shutil.rmtree('AutogluonModels')

gc.collect()